In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("C:/Users/Amir/Downloads/my_train.csv")

In [3]:
train.head()

,Survived,Pclass,SibSp,Parch,FamilySize,Sex_0,Sex_1,Title_Master,Title_Miss,Title_Mr,...,Age_type_Teenage,Age_type_Adult,Age_type_Elder,Em_type_0,Em_type_1,Em_type_2,Fare_type_Low_fare,Fare_type_median_fare,Fare_type_Average_fare,Fare_type_high_fare
0,0,3,1,0,2,True,False,False,False,True,...,False,True,False,True,False,False,True,False,False,False
1,1,1,1,0,2,False,True,False,False,False,...,False,True,False,False,True,False,False,False,False,True
2,1,3,0,0,1,False,True,False,True,False,...,False,True,False,True,False,False,False,True,False,False
3,1,1,1,0,2,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,True
4,0,3,0,0,1,True,False,False,False,True,...,False,True,False,True,False,False,False,True,False,False


In [4]:
train = train.astype('float32')
train.head()

,Survived,Pclass,SibSp,Parch,FamilySize,Sex_0,Sex_1,Title_Master,Title_Miss,Title_Mr,...,Age_type_Teenage,Age_type_Adult,Age_type_Elder,Em_type_0,Em_type_1,Em_type_2,Fare_type_Low_fare,Fare_type_median_fare,Fare_type_Average_fare,Fare_type_high_fare
0,0.0,3.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
train.shape


(891, 23)

In [6]:
# Filter rows where the 'Survived' column is 0 (i.e., passengers who did not survive)
df_zeros = train[train['Survived'] == 0]  # Create a DataFrame containing passengers who did not survive

# Filter rows where the 'Survived' column is 1 (i.e., passengers who survived)
df_ones = train[train['Survived'] == 1]  # Create a DataFrame containing passengers who survived


In [7]:
del df_zeros['Survived']
del df_ones['Survived']

In [8]:
# Import necessary layers and libraries for building the generator model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
import numpy as np

# Define the dimensions for the input and output
latent_dim = 100  # Dimension of the noise space (input to the generator)
input_dim = df_zeros.shape[1]  # Number of features in the dataframe (output size of the generator)

# Define the Generator model
def build_generator():
    # Input layer, representing the noise vector with latent_dim dimensions
    generator_input = Input(shape=(latent_dim,))
    
    # First hidden layer with 128 units and ReLU activation
    x = Dense(128, activation='relu')(generator_input)
    
    # Second hidden layer with 64 units and ReLU activation
    x = Dense(64, activation='relu')(x)
    
    # Output layer with the same number of units as input_dim (features of the data)
    # Sigmoid activation is used to constrain the output between 0 and 1
    x = Dense(input_dim, activation='sigmoid')(x)
    
    # Define the model with the input and output layers
    generator_model = Model(generator_input, x)
    
    return generator_model

# Build and summarize the generator model
generator = build_generator()
generator.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 22)                  │           1,430 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,614 (88.34 KB)

 Trainable params: 22,614 (88.34 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Import necessary layers and libraries for building the discriminator model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

# Define the Discriminator model
def build_discriminator():
    # Input layer for the discriminator model, which takes the generated data as input
    discriminator_input = Input(shape=(input_dim,))
    
    # First hidden layer with 64 units and LeakyReLU activation
    x = Dense(64)(discriminator_input)
    x = LeakyReLU(alpha=0.2)(x)  # LeakyReLU activation with a small slope for negative values
    
    # Second hidden layer with 32 units and LeakyReLU activation
    x = Dense(32)(x)
    x = LeakyReLU(alpha=0.2)(x)  # LeakyReLU activation
    
    # Output layer with 1 unit and sigmoid activation function (binary classification: real or fake)
    x = Dense(1, activation='sigmoid')(x)
    
    # Define the discriminator model with the input and output layers
    discriminator_model = Model(discriminator_input, x)
    
    # Compile the discriminator model with Adam optimizer and binary crossentropy loss
    discriminator_model.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])
    
    return discriminator_model

# Build and summarize the discriminator model
discriminator = build_discriminator()
discriminator.summary()


C:\Users\Amir\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 22)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           1,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,585 (14.00 KB)

 Trainable params: 3,585 (14.00 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Import the necessary libraries for building the GAN model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input

# Define the GAN model
def build_gan(generator, discriminator):
    # Freeze the discriminator during the training of the GAN model
    discriminator.trainable = False  # When training the GAN, we freeze the discriminator
    
    # Input layer for the GAN, which takes random noise as input
    gan_input = Input(shape=(latent_dim,))
    
    # Generate data using the generator model
    generated_data = generator(gan_input)
    
    # Pass the generated data through the discriminator to classify it as real or fake
    gan_output = discriminator(generated_data)
    
    # Define the GAN model with the input and output layers
    gan = Model(gan_input, gan_output)
    
    # Compile the GAN model with Adam optimizer and binary crossentropy loss function
    gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')
    
    return gan

# Build and summarize the GAN model
gan = build_gan(generator, discriminator)
gan.summary()


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional_13 (Functional)           │ (None, 22)                  │          22,614 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional_14 (Functional)           │ (None, 1)                   │           3,585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,199 (102.34 KB)

 Trainable params: 22,614 (88.34 KB)

 Non-trainable params: 3,585 (14.00 KB)

In [ ]:
import tensorflow as tf
import numpy as np

# Define the GAN training function
def train_gan(generator, discriminator, gan, data, epochs=5000, batch_size=32):
    for epoch in range(epochs):
        # Select a random batch of real data
        idx = np.random.randint(0, data.shape[0], batch_size)  # Random indices for selecting real data
        real_data = data[idx]  # Real data batch

        # Generate fake data using the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Random noise as input for the generator
        generated_data = generator.predict(noise)  # Generate fake data

        # Combine real and fake data
        X = np.vstack((real_data, generated_data))  # Combine real and fake data into one batch
        y = np.hstack((np.ones((batch_size,)), np.zeros((batch_size,))))  # Labels: 1 for real, 0 for fake

        # Train the Discriminator
        d_loss = discriminator.train_on_batch(X, y)  # Train the discriminator on real and fake data

        # Generate noise for training the GAN
        noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Random noise for GAN training
        misleading_targets = np.ones((batch_size,))  # Misleading labels: target the generator to output real data

        # Train the Generator through the GAN model
        g_loss = gan.train_on_batch(noise, misleading_targets)  # Train the generator to fool the discriminator

        # Display progress every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D loss: {d_loss[0]}, G loss: {g_loss}")

# Train the GAN model
train_gan(generator, discriminator, gan, df_zeros.values, epochs=5000, batch_size=32)


In [35]:
# Generate new data 
num_new_samples = 500  # For example, generate 500 new samples

# Generate random noise
noise = np.random.normal(0, 1, (num_new_samples, latent_dim))  # Random noise input for the generator

# Generate new data using the Generator
new_data = generator.predict(noise)  # Generate new data from the noise

# Since the output is continuous in the range (0, 1), you can convert it to binary (0 or 1)
# Using a threshold of 0.5
new_data_binary = (new_data > 0.5).astype(int)  # Convert to binary based on the threshold

# Display the new data
import pandas as pd

# Create a DataFrame from the new binary data
new_data_zeros = pd.DataFrame(new_data_binary, columns=[f'feature_{i}' for i in range(new_data_binary.shape[1])])
print(new_data_zeros.head())  # Display the first few rows of the new data


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0          0          0          1          0          0          1   
1          0          0          1          0          0          1   
2          0          0          1          0          0          1   
3          0          0          1          0          0          1   
4          0          0          1          0          0          1   

   feature_6  feature_7  feature_8  feature_9  ...  feature_13  feature_14  \
0          1          1          1          1  ...           0           1   
1          1          1          1          1  ...           0           1   
2          1          1          1          1  ...           0           1   
3          1          1          1          1  ...           0           1   
4          1          1          1          1  ...           0           1   

   feature_15  feature_16  feature_17  feature_18  feature_19  f

In [40]:
new_data_zeros.head(50)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22
0,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
1,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
2,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
3,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
4,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
5,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
6,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
7,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
8,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1
9,0,0,1,0,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,1,1


In [ ]:
#_________________________________________________________
#-------------------------------------------------------------------------
#------------------------------------------------------------------------------
























#----------------------------------------------------------------------------
#----------------------------------------------------------------------------------

In [19]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Input
from keras.optimizers import Adam

# Set input dimensions and number of features
latent_dim = 10  # Latent space dimension (noise input)
n_features = 22  # Number of features in your dataset (df_zeros)

# Generator model
def build_generator():
    model = Sequential()
    
    # First layer: takes noise input and transforms it into a hidden layer
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))  # LeakyReLU activation
    model.add(BatchNormalization(momentum=0.8))  # BatchNormalization to stabilize training

    # Second layer
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))  # LeakyReLU activation
    model.add(BatchNormalization(momentum=0.8))  # BatchNormalization

    # Output layer: generates synthetic data with n_features (number of features)
    model.add(Dense(n_features, activation='sigmoid'))  # Sigmoid activation to get values between 0 and 1
    
    return model

# Discriminator model
def build_discriminator():
    model = Sequential()
    
    # Input layer that takes in the data
    model.add(Dense(256, input_dim=n_features))
    model.add(LeakyReLU(alpha=0.2))  # LeakyReLU activation
    
    # Second layer
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))  # LeakyReLU activation
    
    # Output layer: decides whether the data is real or fake
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation to output probability (real or fake)
    
    return model

# Compile models
optimizer = Adam(0.0002, 0.5)  # Adam optimizer with learning rate 0.0002 and beta 0.5

# Build and compile the discriminator model
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Build the generator model
generator = build_generator()

# GAN model: takes noise as input and sends it to the discriminator for classification
z = Input(shape=(latent_dim,))  # Noise input
generated_data = generator(z)  # Generate data from noise
discriminator.trainable = False  # Freeze the discriminator's weights while training the GAN
validity = discriminator(generated_data)  # Check if the generated data is valid

# Build and compile the GAN model
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)


C:\Users\Amir\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Amir\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [21]:
# Training the GAN
def train_gan(generator, discriminator, gan, data, epochs, batch_size=16):
    half_batch = int(batch_size / 2)  # Half batch size for real and fake data

    for epoch in range(epochs):
        # --- Training the Discriminator ---
        # Real data
        idx = np.random.randint(0, data.shape[0], half_batch)  # Randomly sample real data
        real_data = data.iloc[idx]  # Get real data from df_zeros

        # Fake data
        noise = np.random.normal(0, 1, (half_batch, latent_dim))  # Generate noise
        fake_data = generator.predict(noise)  # Generate fake data using the generator

        # Train the discriminator on real and fake data
        d_loss_real = discriminator.train_on_batch(real_data, np.ones((half_batch, 1)))  # Label real data as 1
        d_loss_fake = discriminator.train_on_batch(fake_data, np.zeros((half_batch, 1)))  # Label fake data as 0
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # Average the loss

        # --- Training the Generator ---
        noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Generate new noise
        valid_y = np.ones((batch_size, 1))  # Label fake data as real (1) for training the generator

        # Train the generator through the GAN model
        g_loss = gan.train_on_batch(noise, valid_y)  # Update the generator to fool the discriminator

        # Print progress every 100 epochs
        if epoch % 100 == 0:
            print(f"{epoch}: [D loss: {d_loss[0]}] [G loss: {g_loss}]")

# Assume df_zeros is your dataframe containing the training data
train_gan(generator, discriminator, gan, df_zeros, epochs=1000)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step


C:\Users\Amir\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0: [D loss: 0.6703634262084961] [G loss: [array(0.691804, dtype=float32), array(0.691804, dtype=float32), array(0.4375, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [19]:
batch_size = 16  # or any other desired value

# Generate random noise
noise = np.random.normal(0, 1, (1000, latent_dim))

# Generate fake data using the generator
generated_data = generator.predict(noise)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [21]:
# Set the display format for numbers to normal
np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.4f}'.format

# Create a DataFrame from the generated data
generated_df = pd.DataFrame(generated_data)

# Display the first few rows of the generated data
generated_df.head()


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.4839,0.9852,0.0333,0.3110,0.0895,0.3883,0.9837,0.9888,0.1722,0.9985,...,0.1512,0.8323,0.9948,0.0891,0.9791,0.9826,0.7519,0.9615,0.9714,0.9934
1,0.9799,0.3432,0.1300,0.9880,0.0005,0.0015,0.9996,0.4103,0.9859,0.9391,...,0.0007,0.0134,0.7264,0.9995,0.8806,0.9262,0.1518,0.9674,0.9053,0.7077
2,0.0003,0.9949,0.0104,0.0000,1.0000,1.0000,0.0000,0.5952,0.0000,0.9474,...,1.0000,1.0000,0.9399,0.0000,0.6028,0.9537,0.9999,0.6539,0.9557,0.9438
3,0.0785,0.8146,0.9454,0.9981,0.0250,0.0009,0.9962,0.9874,0.8872,0.9865,...,0.1653,0.0899,0.9858,0.6708,0.9397,0.9642,0.0474,0.9770,0.2566,0.9611
4,0.9757,0.0001,0.9904,1.0000,0.0000,0.0000,1.0000,0.8475,1.0000,0.0268,...,0.0000,0.0000,0.0019,1.0000,0.6948,0.9706,0.0000,0.9781,0.0156,0.0165


In [23]:
# Add a 'Survived' column with a constant value of 0
generated_df['Survived'] = 0

# Move the 'Survived' column to the beginning of the DataFrame
cols = ['Survived'] + [col for col in generated_df.columns if col != 'Survived']
generated_df = generated_df[cols]

# Display the data to ensure the changes
generated_df.head()


,Survived,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,0,0.4839,0.9852,0.0333,0.3110,0.0895,0.3883,0.9837,0.9888,0.1722,...,0.1512,0.8323,0.9948,0.0891,0.9791,0.9826,0.7519,0.9615,0.9714,0.9934
1,0,0.9799,0.3432,0.1300,0.9880,0.0005,0.0015,0.9996,0.4103,0.9859,...,0.0007,0.0134,0.7264,0.9995,0.8806,0.9262,0.1518,0.9674,0.9053,0.7077
2,0,0.0003,0.9949,0.0104,0.0000,1.0000,1.0000,0.0000,0.5952,0.0000,...,1.0000,1.0000,0.9399,0.0000,0.6028,0.9537,0.9999,0.6539,0.9557,0.9438
3,0,0.0785,0.8146,0.9454,0.9981,0.0250,0.0009,0.9962,0.9874,0.8872,...,0.1653,0.0899,0.9858,0.6708,0.9397,0.9642,0.0474,0.9770,0.2566,0.9611
4,0,0.9757,0.0001,0.9904,1.0000,0.0000,0.0000,1.0000,0.8475,1.0000,...,0.0000,0.0000,0.0019,1.0000,0.6948,0.9706,0.0000,0.9781,0.0156,0.0165


In [25]:
# Training the GAN for 1 epoch
def train_gan(generator, discriminator, gan, data, epochs, batch_size=16):
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
        # --- Training the Discriminator ---
        # Real data
        idx = np.random.randint(0, data.shape[0], half_batch)
        real_data = data.iloc[idx]

        # Fake data
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_data = generator.predict(noise)

        # Training the Discriminator: real and fake data
        d_loss_real = discriminator.train_on_batch(real_data, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_data, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # --- Training the Generator ---
        # Fake data with real labels
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))

        # Training the GAN: the goal is to generate data that tricks the Discriminator
        g_loss = gan.train_on_batch(noise, valid_y)

        # Displaying progress
        if epoch % 100 == 0:
            print(f"{epoch}: [D loss: {d_loss[0]}] [G loss: {g_loss}]")

# Assume df_zeros is your DataFrame.
train_gan(generator, discriminator, gan, df_ones, epochs=1000)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0: [D loss: 0.8538435697555542] [G loss: [array(0.8539152, dtype=float32), array(0.8539152, dtype=float32), array(0.49775225, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━

In [27]:
noise

array([[ 0.45347967, -1.01194002,  1.71880442, ..., -0.34584795,
         0.87557451,  1.06741326],
       [-0.21299968,  2.56603659, -1.26132927, ...,  0.35711751,
        -0.08771298, -0.0485858 ],
       [-0.71586644, -0.33962376, -0.17911752, ...,  0.74236466,
        -0.01237377, -0.69129204],
       ...,
       [ 0.65262256,  1.8657517 , -0.48915608, ..., -0.56686434,
        -0.35588012, -0.87944348],
       [ 2.10920194, -1.27032838, -0.45471261, ..., -1.06889213,
         1.1187963 ,  0.39057412],
       [-0.45242429,  1.40720225,  0.52451963, ...,  0.15191353,
        -2.14497224,  0.77995301]])

In [29]:
generated_data_ones = generator.predict(noise)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [35]:
# تنظیم نمایش اعداد به فرمت معمولی
np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.4f}'.format

generated_ones = pd.DataFrame(generated_data_ones)
# نمایش داده‌های تولید شده
generated_ones.head() , generated_ones.shape

(      0      1      2      3      4      5      6      7      8      9   ...  \
 0 0.3114 0.9965 0.0014 0.0026 0.4222 0.9925 0.9260 0.9972 0.0014 0.9980  ...   
 1 0.9975 0.9982 0.0053 0.5529 0.0003 0.0507 1.0000 0.9924 0.6706 0.9986  ...   
 2 0.0000 0.9970 0.0014 0.0000 1.0000 1.0000 0.0000 0.9300 0.0000 0.9995  ...   
 3 0.8260 0.9946 0.0037 0.0683 0.1467 0.7996 0.9886 0.9968 0.0255 0.9979  ...   
 4 0.9997 0.9974 0.0732 1.0000 0.0000 0.0000 1.0000 0.9981 1.0000 0.9995  ...   
 
       12     13     14     15     16     17     18     19     20     21  
 0 0.0998 0.9985 0.9967 0.0011 0.9579 0.9977 0.9899 0.9951 0.9972 0.9960  
 1 0.0001 0.9924 0.9983 0.9276 0.9961 0.9948 0.9982 0.9946 0.9947 0.9982  
 2 1.0000 0.9936 0.9969 0.0000 0.4332 0.9990 0.9620 0.9950 0.9957 0.9973  
 3 0.0250 0.9960 0.9979 0.0285 0.9754 0.9956 0.9955 0.9976 0.9968 0.9971  
 4 0.0000 0.0768 0.9984 1.0000 0.9992 0.9980 0.9844 0.9975 0.9952 0.9993  
 
 [5 rows x 22 columns],
 (3000, 22))

In [37]:
# اضافه کردن ستون 'Survived' با مقدار ثابت 0
generated_ones['Survived'] = 1

# جابجا کردن ستون 'Survived' به ابتدای دیتافریم
cols = ['Survived'] + [col for col in generated_ones.columns if col != 'Survived']
generated_ones = generated_ones[cols]

# نمایش داده‌ها برای اطمینان
generated_ones.head()

,Survived,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,1,0.3114,0.9965,0.0014,0.0026,0.4222,0.9925,0.9260,0.9972,0.0014,...,0.0998,0.9985,0.9967,0.0011,0.9579,0.9977,0.9899,0.9951,0.9972,0.9960
1,1,0.9975,0.9982,0.0053,0.5529,0.0003,0.0507,1.0000,0.9924,0.6706,...,0.0001,0.9924,0.9983,0.9276,0.9961,0.9948,0.9982,0.9946,0.9947,0.9982
2,1,0.0000,0.9970,0.0014,0.0000,1.0000,1.0000,0.0000,0.9300,0.0000,...,1.0000,0.9936,0.9969,0.0000,0.4332,0.9990,0.9620,0.9950,0.9957,0.9973
3,1,0.8260,0.9946,0.0037,0.0683,0.1467,0.7996,0.9886,0.9968,0.0255,...,0.0250,0.9960,0.9979,0.0285,0.9754,0.9956,0.9955,0.9976,0.9968,0.9971
4,1,0.9997,0.9974,0.0732,1.0000,0.0000,0.0000,1.0000,0.9981,1.0000,...,0.0000,0.0768,0.9984,1.0000,0.9992,0.9980,0.9844,0.9975,0.9952,0.9993


In [39]:
# اضافه کردن ستون‌های مفقود در دیتافریم‌های GAN
for col in train.columns:
    if col not in generated_ones.columns:
        generated_ones[col] = 0  # یا یک مقدار پیش‌فرض مناسب مثل np.nan
    if col not in generated_df.columns:
        generated_df[col] = 0

# مرتب‌سازی ستون‌ها به ترتیب دیتافریم اصلی
generated_ones = generated_ones[train.columns]
generated_df = generated_df[train.columns]

# اتصال دیتافریم‌های GAN به انتهای دیتافریم اصلی
df_combined = pd.concat([train, generated_ones, generated_df], ignore_index=True)


In [41]:
df_combined.head()

,Survived,Pclass,SibSp,Parch,FamilySize,Sex_0,Sex_1,Title_Master,Title_Miss,Title_Mr,...,Age_type_Teenage,Age_type_Adult,Age_type_Elder,Em_type_0,Em_type_1,Em_type_2,Fare_type_Low_fare,Fare_type_median_fare,Fare_type_Average_fare,Fare_type_high_fare
0,0.0000,3.0000,1.0000,0.0000,2.0000,1.0000,0.0000,0.0000,0.0000,1.0000,...,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
1,1.0000,1.0000,1.0000,0.0000,2.0000,0.0000,1.0000,0.0000,0.0000,0.0000,...,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000
2,1.0000,3.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,1.0000,0.0000,...,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
3,1.0000,1.0000,1.0000,0.0000,2.0000,0.0000,1.0000,0.0000,0.0000,0.0000,...,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
4,0.0000,3.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,1.0000,...,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000


In [43]:
# اتصال دیتاست‌های GAN به انتهای دیتاست اصلی
df_combined = pd.concat([train, generated_df, generated_ones], ignore_index=True)

In [45]:
df_combined.to_csv('train_dataset_GAN.csv', index=False)


In [47]:
#ta akhar rafti qable ke beri biron